In [1]:
import torch.nn as nn
from model_numba.Layers.Layers import *
import numpy as np
from torchvision import transforms
from model_numba.unet_cuda import Double_Conv, Unet_Cuda
from PIL import Image
import torch
import time
from numba import cuda

In [2]:
transform = transforms.Compose(
    [
        transforms.Resize((512, 512)),
        transforms.ToTensor()
    ]
)
image = Image.open('test.jpg')
print(image.size)
image = transform(image).unsqueeze(0)
image.shape

(500, 300)


torch.Size([1, 3, 512, 512])

In [3]:
from model.unet import U_net
model = U_net(1).eval()
weight = torch.load('weights/weights.pth')
model.load_state_dict(weight)

numba_model = Unet_Cuda(1).eval()

In [4]:
out = model(image)

In [5]:
def batch_norm_2d(x, gamma, beta, mean, var, eps=1e-5):
    """
    Perform 2D Batch Normalization on NCHW formatted data using provided mean and variance.
    This implementation is purely sequential and avoids multiprocessing.
    
    Parameters:
    - x: Input data of shape (N, C, H, W)
    - gamma: Scale parameter of shape (C,)
    - beta: Shift parameter of shape (C,)
    - mean: Provided mean of shape (C,)
    - var: Provided variance of shape (C,)
    - eps: Small constant for numerical stability
    
    Returns:
    - Normalized data of shape (N, C, H, W)
    """
    # Initialize output tensor
    N, C, H, W = x.shape
    out = np.zeros_like(x)
    
    # Normalize each channel sequentially
    for c in range(C):
        for n in range(N):
            for h in range(H):
                for w in range(W):
                    x_normalized = (x[n, c, h, w] - mean[c]) / np.sqrt(var[c] + eps)
                    out[n, c, h, w] = gamma[c] * x_normalized + beta[c]
    
    return out

In [6]:
N =1
C = 3 
H = 512 
W = 512

input = np.random.uniform(-2.0, 2.0, (N,C,H,W))
weight = np.random.uniform(0.0, 1.0, (C))
bias = np.random.uniform(0.0, 1.0, (C))
mean = np.random.uniform(0.0, 1.0, (C))
var = np.random.uniform(0.0, 1.0, (C))
start = time.time()
out_cpu = batch_norm_2d(input, weight, bias, mean, var)
end = time.time()

print(f'Cpu Runtime: {end - start}')

start = time.time()
out_numba = batchNorm2D(input, weight, bias, mean, var)
end = time.time()
print(f'Numba Runtime: {end - start}')

print(f'Mean Difference: {np.mean(np.abs(out_cpu - out_numba)) : .6f}')


Cpu Runtime: 0.7149941921234131
Numba Runtime: 0.22329187393188477
Mean Difference:  0.000000
